In [1]:
import sqlite3
import pandas as pd
from IPython.display import display, clear_output

In [2]:
db_path = 'Chinook_Sqlite.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [3]:
def sql_command(command):
    cursor.execute(command)
    if cursor.description:
        column_names = [description[0] for description in cursor.description]
        display(pd.DataFrame(cursor.fetchall(), columns = column_names))

In [4]:
sql_command("SELECT * FROM Customer LIMIT 5;")
sql_command('select * from invoice LIMIT 5;')
sql_command('select * from invoiceLine LIMIT 5;')
sql_command('select * from Track LIMIT 5;')

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


# Requirment 1:

In [5]:
sql_command('''
            with invoiceCountItems as (
                select 
                    invoice.*,
                    count(invoiceline.InvoiceId) as NumberOfItems
                from 
                    invoice
                left join
                    invoiceline on invoice.invoiceid = invoiceline.invoiceid
                GROUP BY 
                    invoice.InvoiceId
            ),
            profit_per_customer as (
                select 
                    customer.CustomerId, 
                    concat( customer.FirstName, ' ', customer.LastName) as FullName,
                    count(invoiceCountItems.CustomerId) as NumberOfInvoices,
                    sum(invoiceCountItems.Total) as TotalPerCustomer,
                    sum(invoiceCountItems.NumberOfItems) as NumberOfItems
                from 
                    customer
                inner join 
                    invoiceCountItems on customer.CustomerId = invoiceCountItems.CustomerId
                GROUP BY customer.CustomerId
                )
            
            select * from profit_per_customer
            order by TotalPerCustomer desc
            limit 10
            ;''')

,CustomerId,FullName,NumberOfInvoices,TotalPerCustomer,NumberOfItems
0,6,Helena Holý,7,49.62,38
1,26,Richard Cunningham,7,47.62,38
2,57,Luis Rojas,7,46.62,38
3,45,Ladislav Kovács,7,45.62,38
4,46,Hugh O'Reilly,7,45.62,38
5,24,Frank Ralston,7,43.62,38
6,28,Julia Barnett,7,43.62,38
7,37,Fynn Zimmermann,7,43.62,38
8,7,Astrid Gruber,7,42.62,38
9,25,Victor Stevens,7,42.62,38


### Requirment 2:

In [6]:
sql_command(""" with invoiceline_trackname as (
                        select 
                            track.trackid, 
                            track.name,
                            sum(invoiceline.unitprice * invoiceline.quantity ) as Total,
                            count(invoiceline.quantity) as quantitysold
                        from 
                            invoiceline
                        inner join 
                            track on invoiceline.TrackId = track.TrackId
                        group by 
                            track.trackid
                    )
                select 
                    TrackId, 
                    Name, 
                    Total,
                    quantitysold,
                    rank() over(order by quantitysold desc)
                                as salesRank
                from 
                    invoiceline_trackname
                order by salesRank asc
                LIMIT 10;
            """)

,trackid,name,Total,quantitysold,salesRank
0,2,Balls to the Wall,1.98,2,1
1,8,Inject The Venom,1.98,2,1
2,9,Snowballed,1.98,2,1
3,20,Overdose,1.98,2,1
4,32,Deuces Are Wild,1.98,2,1
5,48,Not The Doctor,1.98,2,1
6,66,Por Causa De Você,1.98,2,1
7,84,Welcome Home (Sanitarium),1.98,2,1
8,161,Snowblind,1.98,2,1
9,162,Cornucopia,1.98,2,1


### Requirment 3:

In [7]:
db_path = 'Chinook_Sqlite.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
sql_command(''' 
    explain 
    select 
        customerid,
        sum(total) as totalsales
    from 
        invoice
    group by 
        customerid;
            ''')
sql_command(''' 
    create index if not exists idx_invoice_customerid on invoice(customerid);
            ''')
sql_command(''' 
    explain 
    select 
        customerid,
        sum(total) as totalsales
    from 
        invoice
    group by 
        customerid;
            ''')

,addr,opcode,p1,p2,p3,p4,p5,comment
0,0,Init,0,37,0,None,0,None
1,1,Noop,1,2,0,None,0,None
2,2,Integer,0,2,0,None,0,None
3,3,Null,0,5,5,None,0,None
4,4,Gosub,4,33,0,None,0,None
5,5,OpenRead,0,396,0,9,0,None
6,6,OpenRead,2,2,0,"k(2,,)",0,None
7,7,Rewind,2,22,7,0,0,None
8,8,DeferredSeek,2,0,0,None,0,None
9,9,Column,2,0,6,None,0,None


,addr,opcode,p1,p2,p3,p4,p5,comment
0,0,Init,0,37,0,None,0,None
1,1,Noop,1,2,0,None,0,None
2,2,Integer,0,2,0,None,0,None
3,3,Null,0,5,5,None,0,None
4,4,Gosub,4,33,0,None,0,None
5,5,OpenRead,0,396,0,9,0,None
6,6,OpenRead,2,2,0,"k(2,,)",0,None
7,7,Rewind,2,22,7,0,0,None
8,8,DeferredSeek,2,0,0,None,0,None
9,9,Column,2,0,6,None,0,None
